In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Term_Project_Unsupervised_Learners/data/"
path = "/content/drive/My Drive/Term_Project_Unsupervised_Learners/data/"

 aggregated_1119.csv		    'aggregated_us_only (1).csv'
'aggregated_aggregated (1).gsheet'   aggregated_us_only.csv
 aggregated_aggregated.csv	     credits_new.csv
 aggregated_aggregated.gsheet	     movie_new.csv
 aggregated.csv			     original_data
 aggregated_new.csv		     oscars.csv
 aggregated_pd_companies.csv


In [ ]:
credits = pd.read_csv(path+"credits.csv")
credits.dtypes
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [ ]:
movie = pd.read_csv(path+"movies_metadata.csv")
movie.dtypes

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [ ]:
# convert data type to more reasonable ones for each column
for i in ['id','budget','popularity']:
    movie.drop(movie[pd.to_numeric(movie[i], errors='coerce').isnull()].index,inplace= True)
    movie[i] = pd.to_numeric(movie[i])
movie['release_date'] = pd.to_datetime(movie['release_date'])
string_ls = ['homepage','imdb_id','original_language','original_title','overview','poster_path','status','tagline','title']
movie[string_ls] = movie[string_ls].astype('string')
movie.dtypes

adult                            object
belongs_to_collection            object
budget                            int64
genres                           object
homepage                         string
id                                int64
imdb_id                          string
original_language                string
original_title                   string
overview                         string
popularity                      float64
poster_path                      string
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           string
tagline                          string
title                            string
video                            object
vote_average                    float64
vote_count                      float64
dtype: object

In [ ]:
# drop duplicated rows
print(movie.shape,credits.shape)
movie.drop_duplicates(inplace=True,subset=['id'])
credits.drop_duplicates(inplace=True,subset=['id'])
print(movie.shape,credits.shape)

(45460, 24) (45476, 3)
(45430, 24) (45432, 3)


In [ ]:
# drop rows not in the other dataset
print(movie.id.size,credits.id.size)
unq_mov,ct_mov = np.unique(np.array(movie.id),return_counts = True)
unq_crd,ct_crd = np.unique(np.array(credits.id),return_counts = True)
crd = np.setdiff1d(unq_crd,unq_mov)
mov = np.setdiff1d(unq_mov,unq_crd)
print(mov,crd)
movie_new = movie.set_index('id').drop(mov)
credits_new = credits.set_index('id').drop(crd)
print(movie_new.shape,credits_new.shape)


45430 45432
[401840] [ 82663 122662 249260]
(45429, 23) (45429, 2)


In [ ]:
movie_new.to_csv(path+'movie_new.csv',index_label = 'id')
credits_new.to_csv(path+'credits_new.csv',index_label = 'id')

In [ ]:
new_ds = movie_new.join(credits_new)
print(new_ds.shape)
print(new_ds.dtypes)

(45429, 25)
adult                            object
belongs_to_collection            object
budget                            int64
genres                           object
homepage                         string
imdb_id                          string
original_language                string
original_title                   string
overview                         string
popularity                      float64
poster_path                      string
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           string
tagline                          string
title                            string
video                            object
vote_average                    float64
vote_count                      float64
cast                             object
crew                        

In [ ]:
# json_ls = ['cast','crew','production_companies','production_countries','spoken_languages','genres','belongs_to_collection']
new_ds.to_csv(path+'aggregated.csv',index_label = 'id')
